In [75]:
#Dashboard: 
#high level metrics of survey demographis
    #Number of Respondents, years distribution (may have to parse this and pick numbers), ticket usage
#Avg rep scores
#Benefits (value and utilization)
#Gameday experience - avg ratings, theme analysis
#AF buttons
#Why renew
#Additional feedback

In [119]:
#Imports
import pandas as pd
from gensim import corpora, models
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import spacy
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 1850000
from collections import Counter
from spacy.matcher import Matcher

In [135]:
#connection
conn = snowflake.connector.Connect(user='ANALYST', password='Fedewap2023!', account='rl43903.east-us-2.azure')
sql = "select * from chicagofire_database.salesforce.sf_formstacks where FORM_NAME__C like '2024 Renewal Member Survey'"
res = conn.cursor().execute(sql).fetchall()
#load in dataset
df = pd.read_sql(sql, conn)

In [78]:
# Assuming df is your DataFrame and 'benefits_column' is the column with the benefits
# Split the 'benefits_column' and create a new DataFrame
benefits_list = df['MULTI_VALUE_PICKLIST_1__C'].str.split(';').explode()

# Count the occurrences of each benefit
benefits_count = benefits_list.value_counts()

In [79]:
benefits_count

Complimentary Parking (new for 2024)                                                 220
Flexible Ticket Exchange Program                                                     216
20% Discount on all Concessions & Merchandise                                        153
Member-only Payment Plans                                                            105
Invitation to the 'Meet the Players Event' and other Exclusive Member-only events     87
Pre-sale opportunities for Fire matches & other events                                74
Member-only Entrance (new for 2024)                                                   64
Exclusive 2024 Member gift                                                            47
Early Entrance on Select Matches                                                      23
Member discounts on groups of 10+ and suites                                           9
Name: MULTI_VALUE_PICKLIST_1__C, dtype: int64

In [80]:
benefits_count.to_csv("C:/Users/dgilbert/Desktop/Code/Member Scoring Model/benefitsq2.csv")

In [81]:
df.columns.to_list()

['ATTRIBUTES',
 'ID',
 'NUMBER_OF_TICKETS__C',
 'AD_CAMPAIGN_NAME__C',
 'AD_NAME__C',
 'AD_PLATFORM__C',
 'AD_SET_COLLECTED_DATE__C',
 'AD_SET_NAME__C',
 'AGE_GROUP__C',
 'ALLSTATE_OPT_IN__C',
 'BURNHAM_COMMENTS__C',
 'BURNHAM_DOWNS__C',
 'BURNHAM_ENTERTAINMENT_RATING__C',
 'BURNHAM_FOOD_RATING__C',
 'BURNHAM_MUSIC_RATING__C',
 'STAFF_RATING__C',
 'CASHIER_EXPERIENCE__C',
 'CFFC_CLASS_RATING__C',
 'CFFC_SCHEDULE_MULTI_SELECT__C',
 'CFRS_ADD_ON__C',
 'CHECKBOX_1__C',
 'CHECKBOX_2__C',
 'CHECKBOX_3__C',
 'CHECKBOX_4__C',
 'CHILD_SOCCER_ATTITUDE__C',
 'COACHING_QUALITY_RATING__C',
 'COMMENTS__C',
 'COMMUNICATION_RATING__C',
 'COMPETITION_QUALITY_RATING__C',
 'CONCESSIONS__C',
 'CONCESSIONS_COMMENTS__C',
 'CONCESSIONS_FOOD_QUALITY_RATING__C',
 'CONCESSIONS_VALUE_RATING__C',
 'CONCESSIONS_VARIETY_RATING__C',
 'CONCESSIONS_WAIT_TIME_RATING__C',
 'CONTACT__C',
 'COVID_19_SAFETY_RATING__C',
 'CREATEDBYID',
 'CURRENCY_1__C',
 'CURRENCY_2__C',
 'CURRENT_CLUB__C',
 'DATE_1__C',
 'DATE_2__C',
 'DA

In [124]:
Renewals = ['ID', 'TEXT_1__C', 'SINGLE_VALUE_PICKLIST_1__C', 'SINGLE_VALUE_PICKLIST_2__C', 'FRIENDLINESS_RATING__C', 'RESPONSE_TIME_RATING__C',
            'PROBLEM_SOLVING_RATING__C', 'REP_ATTENTIVENESS__C', 'PATIENCE_RATING__C', 'REP_COMMUNICATION_RATING__C', 'PRODUCT_KNOWLEDGE_RATING__C', 'PROFESSIONALISM__C'
            , 'MULTI_VALUE_PICKLIST_1__C', 'MULTI_VALUE_PICKLIST_2__C', 'SINGLE_VALUE_PICKLIST_4__C', 'CONCESSIONS__C', 'MERCHANDISE__C', 'PARKING_LOT_ARRIVAL_TIME__C', 'PARKING_LOT_DEPARTURE_TIME__C', 
            'NUMBER_1__C', 'NUMBER_2__C', 'PARKING_COMMENTS__C', 'STADIUM_GATE_RATING__C', 'COMMENTS__C', 'CONCESSIONS_WAIT_TIME_RATING__C', 'CONCESSIONS_VARIETY_RATING__C', 
            'NUMBER_3__C', 'CONCESSIONS_VALUE_RATING__C', 'CONCESSIONS_COMMENTS__C', 'MERCHCHANDISE_WAIT_TIME__C', 'MERCHCHANDISE_VARIETY__C', 'MERCHANDISE_VALUE_RATING__C', 'MERCHANDISE_RATING__C', 
            'NUMBER_4__C', 'MERCHANDISE_COMMENTS__C', 'NEW_BENEFITS_RATING__C', 'FAMILY_FRIENDLY_RATING__C', 'SENSE_OF_COMMUNITY__C', 'LONG_TEXT_1__C', 'NPS__C', 'SINGLE_VALUE_PICKLIST_5__C', 'LONG_TEXT_2__C']

In [145]:
renew = df.loc[df['FORM_NAME__C'] == '2024 Renewal Member Survey', Renewals]
renew

,ID,TEXT_1__C,SINGLE_VALUE_PICKLIST_1__C,SINGLE_VALUE_PICKLIST_2__C,FRIENDLINESS_RATING__C,RESPONSE_TIME_RATING__C,PROBLEM_SOLVING_RATING__C,REP_ATTENTIVENESS__C,PATIENCE_RATING__C,REP_COMMUNICATION_RATING__C,...,MERCHANDISE_RATING__C,NUMBER_4__C,MERCHANDISE_COMMENTS__C,NEW_BENEFITS_RATING__C,FAMILY_FRIENDLY_RATING__C,SENSE_OF_COMMUNITY__C,LONG_TEXT_1__C,NPS__C,SINGLE_VALUE_PICKLIST_5__C,LONG_TEXT_2__C
0,a2JUr0000000yYrMAI,8,Personal,Michael Serio,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,I love the unique offerings like the Champions...,2.0,3.0,2.0,I enjoy the matchday atmosphere and the Chicag...,6.0,No,I would appreciate included Open Cup and leagu...
1,a2JUr0000001125MAA,2,Personal,I'm not sure,4.0,5.0,3.0,4.0,3.0,4.0,...,2.0,2.0,None,2.0,2.0,2.0,Want to see fire actually make the playoffs!,6.0,No,I would love more day games! Only having night...
2,a2JUr00000011mrMAA,10,Both,Marina Letine Thorne,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,Sometimes sold out on items or sizes i was loo...,1.0,1.0,2.0,I want to see the Fire succeed and get to the ...,10.0,Yes,None
3,a2JUr00000012PZMAY,26,Personal,Michael Serio,2.0,2.0,2.0,2.0,2.0,2.0,...,None,None,None,2.0,2.0,3.0,Hoping for improved team performance.,0.0,No,Every year that goes by I get more frustrated ...
4,a2JUr00000013jpMAA,3,Personal,I'm not sure,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,None,1.0,1.0,1.0,None,0.0,Yes,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,a2JUr0000000xRVMAY,26,Business,Michael Serio,1.0,1.0,3.0,1.0,1.0,1.0,...,None,None,None,2.0,2.0,3.0,To give out as a benefit to my employees.,7.0,No,None
303,a2JUr000000155hMAA,26,Both,Marina Letine Thorne,2.0,4.0,5.0,3.0,2.0,3.0,...,2.0,2.0,None,3.0,2.0,3.0,Despite the staff lying to me about my selecti...,2.0,No,I was # 27 on the original ticket purchase at ...
304,a2JUr00000015qTMAQ,18,Personal,Marina Letine Thorne,2.0,2.0,3.0,3.0,3.0,2.0,...,None,None,None,2.0,2.0,4.0,Enjoy soccer,1.0,No,The product on the field is very poor and is m...
305,a2JUr0000001ZrmMAE,15,Personal,Adrian Corona,1.0,1.0,1.0,1.0,1.0,1.0,...,None,None,None,1.0,1.0,1.0,None,6.0,Yes,None


In [155]:
# Initialize VADER
sia = SentimentIntensityAnalyzer()

# Function to categorize sentiment
def get_sentiment_scores(text):
    if text is None or text.strip() == '':
        return {'neg': 0, 'neu': 1, 'pos': 0, 'compound': 0}
    return sia.polarity_scores(text)

# Apply sentiment analysis to get raw scores
renew['additional_sentiment_scores'] = renew['LONG_TEXT_2__C'].apply(get_sentiment_scores)
# Separate each sentiment score into its own column
renew['additional_sentiment_neg'] = renew['additional_sentiment_scores'].apply(lambda x: x['neg'])
renew['additional_sentiment_neu'] = renew['additional_sentiment_scores'].apply(lambda x: x['neu'])
renew['additional_sentiment_pos'] = renew['additional_sentiment_scores'].apply(lambda x: x['pos'])
renew['additional_sentiment_compound'] = renew['additional_sentiment_scores'].apply(lambda x: x['compound'])

In [156]:
renew

,ID,TEXT_1__C,SINGLE_VALUE_PICKLIST_1__C,SINGLE_VALUE_PICKLIST_2__C,FRIENDLINESS_RATING__C,RESPONSE_TIME_RATING__C,PROBLEM_SOLVING_RATING__C,REP_ATTENTIVENESS__C,PATIENCE_RATING__C,REP_COMMUNICATION_RATING__C,...,renewal_sentiment_scores,renewal_sentiment_neg,renewal_sentiment_neu,renewal_sentiment_pos,renewal_sentiment_compound,additional_sentiment_scores,additional_sentiment_neg,additional_sentiment_neu,additional_sentiment_pos,additional_sentiment_compound
0,a2JUr0000000yYrMAI,8,Personal,Michael Serio,2.0,2.0,2.0,2.0,2.0,2.0,...,"{'neg': 0.136, 'neu': 0.682, 'pos': 0.182, 'co...",0.136,0.682,0.182,0.2023,"{'neg': 0.0, 'neu': 0.571, 'pos': 0.429, 'comp...",0.000,0.571,0.429,0.8824
1,a2JUr0000001125MAA,2,Personal,I'm not sure,4.0,5.0,3.0,4.0,3.0,4.0,...,"{'neg': 0.269, 'neu': 0.6, 'pos': 0.13, 'compo...",0.269,0.600,0.130,-0.3382,"{'neg': 0.083, 'neu': 0.635, 'pos': 0.282, 'co...",0.083,0.635,0.282,0.8065
2,a2JUr00000011mrMAA,10,Both,Marina Letine Thorne,1.0,1.0,1.0,1.0,1.0,1.0,...,"{'neg': 0.111, 'neu': 0.583, 'pos': 0.306, 'co...",0.111,0.583,0.306,0.7717,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
3,a2JUr00000012PZMAY,26,Personal,Michael Serio,2.0,2.0,2.0,2.0,2.0,2.0,...,"{'neg': 0.0, 'neu': 0.337, 'pos': 0.663, 'comp...",0.000,0.337,0.663,0.7096,"{'neg': 0.092, 'neu': 0.745, 'pos': 0.162, 'co...",0.092,0.745,0.162,0.7512
4,a2JUr00000013jpMAA,3,Personal,I'm not sure,1.0,1.0,1.0,1.0,1.0,1.0,...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,a2JUr0000000xRVMAY,26,Business,Michael Serio,1.0,1.0,3.0,1.0,1.0,1.0,...,"{'neg': 0.0, 'neu': 0.7, 'pos': 0.3, 'compound...",0.000,0.700,0.300,0.4588,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
303,a2JUr000000155hMAA,26,Both,Marina Letine Thorne,2.0,4.0,5.0,3.0,2.0,3.0,...,"{'neg': 0.073, 'neu': 0.637, 'pos': 0.29, 'com...",0.073,0.637,0.290,0.7332,"{'neg': 0.039, 'neu': 0.909, 'pos': 0.052, 'co...",0.039,0.909,0.052,-0.1494
304,a2JUr00000015qTMAQ,18,Personal,Marina Letine Thorne,2.0,2.0,3.0,3.0,3.0,2.0,...,"{'neg': 0.0, 'neu': 0.238, 'pos': 0.762, 'comp...",0.000,0.238,0.762,0.4939,"{'neg': 0.149, 'neu': 0.851, 'pos': 0.0, 'comp...",0.149,0.851,0.000,-0.7003
305,a2JUr0000001ZrmMAE,15,Personal,Adrian Corona,1.0,1.0,1.0,1.0,1.0,1.0,...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000


In [157]:
renew.to_csv("renewalsurvey.csv", index=False)

In [202]:
from collections import Counter

def preprocess_and_count(text):
    if text is None or text.strip() == '':
        return []
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha()]
    excluded_words = {'team', 'ticket', 'tickets', 'none', 'free', 'get', 'fire', 'game', 'season', 'like', 'store', 'stadium', 'gate', 'parking', 'would', 'way', 'get', 'getting', 'also', 'every', 'food', 'merch', 'need', 'merchandise', 'buy'}
    tokens = [word for word in tokens if word not in excluded_words]
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

# Apply Preprocessing and Count Words
renew['processed'] = renew['LONG_TEXT_2__C'].apply(preprocess_and_count)

# Flatten the list of lists and count word frequencies
all_words = [word for sublist in renew['processed'] for word in sublist]
word_freq = Counter(all_words)

# Get the top 15 most frequent valuable words
top_15_words = word_freq.most_common(15)
print(top_15_words)
new_data_df = pd.DataFrame(top_15_words, columns=['Word', 'Count'])
new_data_df['Question'] = 'Additional'

[('year', 34), ('experience', 32), ('great', 31), ('club', 30), ('field', 28), ('games', 27), ('better', 25), ('see', 21), ('people', 19), ('years', 18), ('new', 18), ('time', 18), ('love', 17), ('day', 17), ('fans', 17)]


In [204]:
# Assuming new_data_df is your new DataFrame with similar columns
word_counts_df = pd.concat([word_counts_df, new_data_df], ignore_index=True)

In [206]:
word_counts_df.to_csv("wordcounts.csv", index=False)